# TASK 4: Machine Learning Implementation & Evaluation 
## Predicting Forest Type 

### Final Project, Python for Data Science 
#### Willamette University MSDS 
by Charles Hanks, Carter McMahon, & Cleighton Roberts 

<br>
<br>
<br>

### Table of Contents: 

4.0 Data Preparation <br> 
4.1 Random Forest Classifier <br>
4.2 Support Vector Machine Classifier <br>
4.3 RF vs. SVM Comparison <br>
4.4 Unsupervised Learning: Clustering <br>
4.5 Additional ML techniques <br>

## 4.0 Data Preparation

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, KFold, cross_val_score
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import confusion_matrix, accuracy_score,cohen_kappa_score, roc_auc_score, roc_curve, classification_report, balanced_accuracy_score
from scipy.stats import loguniform, randint
from sklearn.utils import resample
from sklearn.neighbors import KNeighborsClassifier

Loading the dataset: 

In [6]:
ds = pd.read_csv('data/covtype.csv')

Data Preprocessing - center and scaling: 

In [7]:
scale = StandardScaler()

ds.columns.get_loc('Cover_Type') # finding index of dependent variable 

numerical_features = ds.iloc[:,0:10] # subsetting numerical features we want to scale
scaled_num_features =  pd.DataFrame(scale.fit_transform(numerical_features), columns = numerical_features.columns)

ds = pd.concat([scaled_num_features, ds.iloc[:,10:]], axis = 1) #combining scaled numerical features with other variables

Initial Train/Test Split: 

In [8]:

X = ds.drop(ds.columns[-1], axis = 1)
y = ds[ds.columns[-1]]

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 650, test_size = 0.3)

## 4.1 Random Forest Classifier 

### Baseline model

In [ ]:
rf = RandomForestClassifier(random_state=650)

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
rf_pred = rf.predict(X_test)

print(accuracy_score(rf_pred, y_test))
print(balanced_accuracy_score(rf_pred, y_test))
print(cohen_kappa_score(rf_pred, y_test))

# Accuracy: 0.952009133467964
# Balanced acc.: 0.9402627427863057
# Kappa: 0.9225107895423049

### Hyperparameter tuning with GridSearchCV

In [ ]:
param_grid = {
    'min_samples_leaf': [1, 2],
    'min_samples_split': [2, 4],
    'n_estimators': [250]
}

grid_search = GridSearchCV(estimator = rf, 
                           param_grid = param_grid, 
                           cv = 3, 
                           verbose = 2)

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_params_

# {'bootstrap': True,
#  'min_samples_leaf': 1,
#  'min_samples_split': 2,
#  'n_estimators': 250}

In [ ]:
best_rf = grid_search.best_estimator_

In [ ]:
best_rf_pred = best_rf.predict(X_test)

print(accuracy_score(best_rf_pred, y_test))
print(balanced_accuracy_score(best_rf_pred, y_test))
print(cohen_kappa_score(best_rf_pred, y_test))

# Accuracy: 0.9525254727372866
# Balanced acc.: 0.9417587733036588
# Kappa: 0.9233291692648075

## 4.2 Support Vector Machine Classifier

### Baseline Model

The training this data using the standard SVC() function  takes an exceedingly long time. Per the [sci-kit learn documentation](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html):

*"For large datasets consider using LinearSVC or SGDClassifier instead, possibly after a Nystroem transformer or other Kernel Approximation"*

In [9]:
model_lsvc = LinearSVC(random_state = 650)
model_lsvc.fit(X_train,y_train) 

/Users/chanks/anaconda3/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(random_state=650)

In [10]:
#predicting forest type on test data using first linear SVC model:  
lsvc_predict = model_lsvc.predict(X_test)

#print accuracy score: 
print(accuracy_score(lsvc_predict, y_test))
# accuracy = .71


0.7115958325683863


In [11]:
print(cohen_kappa_score(lsvc_predict, y_test))

0.5249588131713108


In [12]:
model_lsvc.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': True,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'loss': 'squared_hinge',
 'max_iter': 1000,
 'multi_class': 'ovr',
 'penalty': 'l2',
 'random_state': 650,
 'tol': 0.0001,
 'verbose': 0}

### Hyperparameter Tuning with RandomizedSearchCV 
We are using RandomizezdSearchCV over GridSearchCV due to size of dataset and limit of computational power. 

In [ ]:
param_dist = {
    'C': loguniform(.0001, 10000),  #choosing a wide range for C 
    'max_iter': randint(1000, 5000) # randomly sampling between 1000 and 5000
}

n_iter_search = 5
random_search = RandomizedSearchCV(model_lsvc, 
                                   param_distributions = param_dist, 
                                   n_iter = n_iter_search, 
                                   n_jobs =4, 
                                   cv = 3, 
                                   random_state = 650)

In [ ]:
random_search.fit(X_train, y_train)     # note - this took 156 minutes to run

In [9]:
#random_search.best_params_ # C=14.367136335397122, max_iter=1902

NameError: name 'random_search' is not defined

In [ ]:
best_lsvc = LinearSVC(C = random_search.best_params_['C'], 
                      max_iter = random_search.best_params_['max_iter'], 
                      random_state = 650)

In [ ]:
best_lsvc.fit(X_train,y_train)


In [ ]:
#predicting forest type on test data using first linear SVC model:  
lsvc_predict2 = best_lsvc.predict(X_test)

In [12]:
print(accuracy_score(lsvc_predict2, y_test)) #this randomized grid search produced results that were comparable to default hp settings. 
# the search space might not have included the best values. 

# accuracy = .71

NameError: name 'lsvc_predict2' is not defined

## 4.3 RF vs. SVM Comparison

## 4.4 Clustering 

In [ ]:
comb_train = pd.concat([X_train, y_train],axis=1)
comb_train.head()

In [ ]:
#we want to run PCA on the whole model so that we can make a scree plot and determine how many componenets are viable


pca=PCA()
pc=pca.fit_transform(comb_train)

In [ ]:
explained_variance_ratio = pca.explained_variance_ratio_

# Plot the scree plot
plt.figure(figsize=(8, 6))
plt.plot(np.arange(1, len(explained_variance_ratio) + 1), explained_variance_ratio, marker='o')
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.title('Scree Plot')
plt.xticks(np.arange(1, len(explained_variance_ratio) + 1))
plt.grid(True)
plt.show()

In [ ]:
#zooming in
plt.figure(figsize=(8, 6))
plt.plot(np.arange(1, len(explained_variance_ratio) + 1), explained_variance_ratio, marker='o')
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.title('Scree Plot')
plt.xticks(np.arange(1, len(explained_variance_ratio) + 1))
plt.grid(True)
plt.xlim(1, 20)
plt.show()

In [ ]:
# this was a tough decision but I think I will go to 7
pca = PCA(n_components=7)

pca.fit(comb_train)
scores_pca = pca.transform(comb_train)

loadings_df = pd.DataFrame(pca.components_, columns=comb_train.columns)

print(loadings_df)

In [ ]:
kpca = KMeans(n_clusters=3,
             init = "k-means++",
             random_state=650)

#apply to da model

kpca.fit(scores_pca)

kpca_x = pd.concat([X_train.reset_index(drop=True),pd.DataFrame(scores_pca)], axis = 1) 
kpca_x['cluster #'] = kpca.labels_

pd.set_option('display.max_columns', None) 
kpca_x.head()

In [ ]:
#now lets visualize4

g = sns.FacetGrid(kpca_x, col="Wilderness_Area4",row="Soil_Type22",hue="cluster #")
g.map(sns.scatterplot, "Elevation", 'Horizontal_Distance_To_Roadways'
      ,alpha=.01
     )

In [ ]:
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

# Extract the columns for plotting
x, y, z = kpca_x.iloc[:, 0], kpca_x.iloc[:, 2], kpca_x.iloc[:, 1]

# Plot the data with color based on the "cluster" column
ax.scatter(x, y, z, c=kpca_x['cluster #'], cmap='viridis', marker='o', label='Data Points')

# Set axis labels and plot title
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
ax.set_title('3D Scatter Plot with Cluster Colors')

# Show the color bar for the cluster labels
#plt.colorbar(label='cluster #')

# Show the plot
plt.legend()
plt.show()

In [ ]:
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

# Extract the columns for plotting
x, z, y  = kpca_x.iloc[:, 3], kpca_x["Elevation"], kpca_x["Slope"]

# Plot the data with color based on the "cluster" column
ax.scatter(x, y, z, c=kpca_x['cluster #'], cmap='viridis', marker='o', label='Data Points')

# Set axis labels and plot title
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
ax.set_title('3D Scatter Plot with Cluster Colors')

# Show the color bar for the cluster labels
#plt.colorbar(label='cluster #')

# Show the plot
plt.legend()
plt.show()

In [ ]:
#repeat this all again but without forest type

pca = PCA(n_components=12)

pca.fit(X_train)
scores_pca = pca.transform(X_train)

kpca = KMeans(n_clusters=6,
             init = "k-means++",
             random_state=650)

#apply to da model

kpca.fit(scores_pca)

kpca_x = pd.concat([X_train.reset_index(drop=True),pd.DataFrame(scores_pca)], axis = 1) 
kpca_x['cluster #'] = kpca.labels_

pd.set_option('display.max_columns', None) 
kpca_x.head()

g = sns.FacetGrid(kpca_x, col="Wilderness_Area4",row="Soil_Type22",hue="cluster #")
g.map(sns.scatterplot, "Elevation", 'Horizontal_Distance_To_Roadways'
      ,alpha=.01
     )


#really this looks like 3 is the best, I can't really differentiate two of them but whatever

In [ ]:
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

# Extract the columns for plotting
x, y, z = kpca_x.iloc[:, 0], kpca_x.iloc[:, 2], kpca_x.iloc[:, 1]

# Plot the data with color based on the "cluster" column
ax.scatter(x, y, z, c=kpca_x['cluster #'], cmap='viridis', marker='o', label='Data Points')

# Set axis labels and plot title
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
ax.set_title('3D Scatter Plot with Cluster Colors')

# Show the color bar for the cluster labels
#plt.colorbar(label='cluster #')

# Show the plot
plt.legend()
plt.show()

## 4.5 Additional ML Techniques 

### KBest Feature Selection
Removing the features with the least impact on model performance

In [1]:
k_best_selector = SelectKBest(k = 30)
k_best_selector.fit(X = X, y = y)
print(X.columns[k_best_selector.get_support()])

NameError: name 'SelectKBest' is not defined

In [2]:

X_train_kbest = k_best_selector.fit_transform(X_train, y_train)
X_test_kbest = k_best_selector.transform(X_test)

NameError: name 'k_best_selector' is not defined

In [ ]:
model_lsvc.fit(X_train_kbest, y_train)

In [ ]:
#predicting using top 30 variables
lsvc_predict3 = model_lsvc.predict(X_test_kbest)

#print accuracy score: 
print(accuracy_score(lsvc_predict3, y_test))
#accuracy = .707, comparable to baseline model 

Conclusion: Taking the top 30 features per KBest Feature Selection produces an equivalent accuracy to that produced by a model taking all variables. 

### Downsampling: 

This dataset is pretty heavily imbalanced, which is likely affecting our model's ability to differentiate between different tree types. Two common aproaches to handling an imbalanced dataset are downsampling and class weighting.



In [22]:
ds.Cover_Type.value_counts()

2    283301
1    211840
3     35754
7     20510
6     17367
5      9493
4      2747
Name: Cover_Type, dtype: int64

As we can see, target classes 1 and 2 make up 495,141 of the rows, or 85% of the data. This is a good example of when accuracy is not a great measure of model performance. 49% of the data is class 2, which means if the model chose class 2 for every single prediction, it would still be correct half of the time. Accuracy can overestimate a model's performance for this reason. 

Cohen's Kappa is a better metric for evaluating models - it takes into account the agreement between what the model predict and the actual target labels, *adjusting for the agreement that could be expected by chance*. In this sense it is a weighted accuracy. 

kappa = (accuracy - random accuracy) / (1 - random accuracy)


Let's downsample each class to equalize how many of each class is present in our training and testing sets.  Like the name suggests, we are subdividing the data per each target class, and then sampling an equal number of observations from each class. In this case, we will randomly sample 2,747 rows from each class, because Cover Type 4 has the fewest observations (2747):

In [25]:
each_tree_type = []
for i in ds['Cover_Type'].unique():
    each_tree_type.append(ds[ds['Cover_Type'] == i]) #now each element in this list is a dataframe containing each class of tree



[        Elevation    Aspect     Slope  Horizontal_Distance_To_Hydrology  \
0       -1.297805 -0.935157 -1.482820                         -0.053767   
1       -1.319235 -0.890480 -1.616363                         -0.270188   
4       -1.301377 -0.988770 -1.616363                         -0.547771   
6       -1.262089 -0.988770 -0.948649                          0.002690   
7       -1.265660 -0.953028 -1.349277                         -0.166682   
...           ...       ...       ...                               ...   
558788  -1.429955  0.262195 -0.147392                         -0.759486   
558964  -1.451385  0.315808 -0.414477                         -0.684210   
558965  -1.447813  0.315808 -0.414477                         -0.547771   
558966  -1.444242  0.297937 -0.414477                         -0.411332   
559142  -1.458528  0.342614 -0.815106                         -0.547771   

        Vertical_Distance_To_Hydrology  Horizontal_Distance_To_Roadways  \
0                      

In [27]:
len(each_tree_type[6]) # index 6 of list each_tree_type has length 2747

2747

In [29]:
downsampled_trees = []

#downsampling each tree type data frame to match the size of the smallest class, Cover Type 4 with 2747 rows
for i in each_tree_type: 
    downsampled_trees.append(resample(i, replace = False, n_samples = len(each_tree_type[6]), random_state = 650))

In [30]:
#now we need to transform this list of data frames back into one dataframe: 
ds_dsamp = pd.concat(downsampled_trees)

ds_dsamp = ds_dsamp.sample(frac = 1, random_state = 650)

In [31]:
#confirming that we now have a balanced dataset: 
ds_dsamp.Cover_Type.value_counts()

1    2747
7    2747
2    2747
4    2747
3    2747
5    2747
6    2747
Name: Cover_Type, dtype: int64

Now we have a balanced dataset. We hypothesize that the model peformance will weaken now that each class has the same proportion of the training and test data (and train time will decrease due to the considerably smaller dataset). 

Train/Test Split with downsampled dataset: 

In [41]:
X = ds_dsamp.drop(ds_dsamp.columns[-1], axis = 1)
y = ds_dsamp[ds_dsamp.columns[-1]]

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 650, test_size = 0.3)

#### Support Vector Machine

In [33]:
model_lsvc.fit(X_train,y_train) 

/Users/chanks/anaconda3/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(random_state=650)

In [34]:

lsvc_predict = model_lsvc.predict(X_test)

print(accuracy_score(lsvc_predict, y_test))

0.6763737216155313


In [35]:
print(cohen_kappa_score(lsvc_predict, y_test))

0.622345392551398


As you can see, the kappa of our model trained on balanced dataset is .62, whereas the original model's kappa was .52. 

#### Random Forest

In [ ]:
rf = RandomForestClassifier(random_state=650,
                            n_estimators=250)

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
rf_eq_pred = rf.predict(X_test)

In [ ]:
print(accuracy_score(rf_eq_pred, y_test))
print(balanced_accuracy_score(rf_eq_pred, y_test))
print(cohen_kappa_score(rf_eq_pred, y_test))

# The model trained on downsampled data
# Accuracy: 0.7513080594822839
# Balanced acc.: 0.6232433828611598
# Kappa: 0.6259294958340503

# vs

# The model trained on the whole dataset
# Accuracy: 0.952009133467964
# Balanced acc.: 0.9402627427863057
# Kappa: 0.9225107895423049

In [ ]:
imbal_conf = confusion_matrix(rf_pred, y_test)
eq_conf = confusion_matrix(rf_eq_pred, y_test)

print('Imbalanced classes model \n')
print(imbal_conf, '\n\n\n')
print('Equalized classes model \n')
print(eq_conf)

# Imbalanced classes model 
# 
# [[59860  2002     3     0    31     5   283]
#  [ 3621 82704   164     0   581   150    41]
#  [    2   163 10254    90    26   399     0]
#  [    0     2    49   695     0    26     0]
#  [   24   106     8     0  2164     1     0]
#  [    7   104   196    24    14  4587     0]
#  [  122    31     0     0     0     0  5765]] 

# Equalized classes model 

# [[49090 15385     0     0     3     0   196]
#  [ 9131 59027    54     0    64    20     2]
#  [   23  1811  8883     4    35   428     0]
#  [    0    45   476   805     0   122     0]
#  [ 1261  6026    82     0  2697    33     2]
#  [  183  2320  1179     0    17  4565     0]
#  [ 3948   498     0     0     0     0  5889]]

### GridSearchCV with Downsampled Dataset

#### Support Vector Machine

Given how much quicker our downsampled dataset is to train a model, let's return to the GridSearchCV technique in hopes of getting better hyperparamter tuning. GridSearchCV is exhaustive, having a larger search space than RandomizedSearchCV.

In [43]:
param_grid = {
    'C': [3,6,9,12,15,18,21],          
    'max_iter': [500,1000,2000,4000]  
}

In [44]:
grid_search_dsamp = GridSearchCV(model_lsvc, param_grid=param_grid, cv=5)
grid_search_dsamp.fit(X_train, y_train)

/Users/chanks/anaconda3/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/chanks/anaconda3/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/chanks/anaconda3/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/chanks/anaconda3/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/chanks/anaconda3/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/chanks/anaconda3/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: Convergenc

GridSearchCV(cv=5, estimator=LinearSVC(random_state=650),
             param_grid={'C': [3, 6, 9, 12, 15, 18, 21],
                         'max_iter': [500, 1000, 2000, 4000]})

In [45]:
grid_search_dsamp.best_params_

{'C': 9, 'max_iter': 2000}

In [47]:
best_lsvc_dsamp = LinearSVC(C = grid_search_dsamp.best_params_['C'], 
                      max_iter = grid_search_dsamp.best_params_['max_iter'], 
                      random_state = 650)

In [49]:
best_lsvc_dsamp.fit(X_train,y_train)

/Users/chanks/anaconda3/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(C=9, max_iter=2000, random_state=650)

In [51]:
best_lsvc_dsamp_predictions = best_lsvc_dsamp.predict(X_test)

In [52]:
print(accuracy_score(best_lsvc_dsamp_predictions, y_test))

0.6762003813485873


In [53]:
print(cohen_kappa_score(best_lsvc_dsamp_predictions, y_test))

0.6221482409289651


Despite these efforts to improve our SVM model performance, the hyperparamter tuning has had little effect. 

### K-Nearest Neighbors 

Train/test split:


In [54]:
X = ds.drop(ds.columns[-1], axis = 1)
y = ds[ds.columns[-1]]

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 650, test_size = 0.3)

In [56]:
knn = KNeighborsClassifier(n_neighbors=5)


In [57]:
knn.fit(X_train,y_train)

KNeighborsClassifier()

In [58]:
knn_predictions = knn.predict(X_test)

In [62]:
print(accuracy_score(knn_predictions, y_test))

0.9249242702404994


KeyboardInterrupt: 